In [ ]:
import pandas as pd
import numpy as np
import os
import warnings
import re
warnings.filterwarnings("ignore")

In [ ]:
meta = pd.read_csv('/content/drive/MyDrive/SSAC/메인 프로젝트/Dataset/movies_metadata.csv')

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
meta.head()    # 전처리 전 movie_meta 데이터

,adult,belongs_to_collection,budget,genres,homepage,id,imdb_id,original_language,original_title,overview,popularity,poster_path,production_companies,production_countries,release_date,revenue,runtime,spoken_languages,status,tagline,title,video,vote_average,vote_count
0,False,"{'id': 10194, 'name': 'Toy Story Collection', ...",30000000,"[{'id': 16, 'name': 'Animation'}, {'id': 35, '...",http://toystory.disney.com/toy-story,862,tt0114709,en,Toy Story,"Led by Woody, Andy's toys live happily in his ...",21.9469,/rhIRbceoE9lR4veEXuwCC2wARtG.jpg,"[{'name': 'Pixar Animation Studios', 'id': 3}]","[{'iso_3166_1': 'US', 'name': 'United States o...",1995-10-30,373554033.0,81.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,NaN,Toy Story,False,7.7,5415.0
1,False,NaN,65000000,"[{'id': 12, 'name': 'Adventure'}, {'id': 14, '...",NaN,8844,tt0113497,en,Jumanji,When siblings Judy and Peter discover an encha...,17.0155,/vzmL6fP7aPKNKPRTFnZmiUfciyV.jpg,"[{'name': 'TriStar Pictures', 'id': 559}, {'na...","[{'iso_3166_1': 'US', 'name': 'United States o...",1995-12-15,262797249.0,104.0,"[{'iso_639_1': 'en', 'name': 'English'}, {'iso...",Released,Roll the dice and unleash the excitement!,Jumanji,False,6.9,2413.0
2,False,"{'id': 119050, 'name': 'Grumpy Old Men Collect...",0,"[{'id': 10749, 'name': 'Romance'}, {'id': 35, ...",NaN,15602,tt0113228,en,Grumpier Old Men,A family wedding reignites the ancient feud be...,11.7129,/6ksm1sjKMFLbO7UY2i6G1ju9SML.jpg,"[{'name': 'Warner Bros.', 'id': 6194}, {'name'...","[{'iso_3166_1': 'US', 'name': 'United States o...",1995-12-22,0.0,101.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,Still Yelling. Still Fighting. Still Ready for...,Grumpier Old Men,False,6.5,92.0
3,False,NaN,16000000,"[{'id': 35, 'name': 'Comedy'}, {'id': 18, 'nam...",NaN,31357,tt0114885,en,Waiting to Exhale,"Cheated on, mistreated and stepped on, the wom...",3.85949,/16XOMpEaLWkrcPqSQqhTmeJuqQl.jpg,[{'name': 'Twentieth Century Fox Film Corporat...,"[{'iso_3166_1': 'US', 'name': 'United States o...",1995-12-22,81452156.0,127.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,Friends are the people who let you be yourself...,Waiting to Exhale,False,6.1,34.0
4,False,"{'id': 96871, 'name': 'Father of the Bride Col...",0,"[{'id': 35, 'name': 'Comedy'}]",NaN,11862,tt0113041,en,Father of the Bride Part II,Just when George Banks has recovered from his ...,8.38752,/e64sOI48hQXyru7naBFyssKFxVd.jpg,"[{'name': 'Sandollar Productions', 'id': 5842}...","[{'iso_3166_1': 'US', 'name': 'United States o...",1995-02-10,76578911.0,106.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,Just When His World Is Back To Normal... He's ...,Father of the Bride Part II,False,5.7,173.0


In [ ]:
meta = meta[['id', 'title', 'genres',  'overview', 'production_companies', 'release_date']] # 필요한 컬럼만 뽑기
meta.head()

,id,title,genres,overview,production_companies,release_date
0,862,Toy Story,"[{'id': 16, 'name': 'Animation'}, {'id': 35, '...","Led by Woody, Andy's toys live happily in his ...","[{'name': 'Pixar Animation Studios', 'id': 3}]",1995-10-30
1,8844,Jumanji,"[{'id': 12, 'name': 'Adventure'}, {'id': 14, '...",When siblings Judy and Peter discover an encha...,"[{'name': 'TriStar Pictures', 'id': 559}, {'na...",1995-12-15
2,15602,Grumpier Old Men,"[{'id': 10749, 'name': 'Romance'}, {'id': 35, ...",A family wedding reignites the ancient feud be...,"[{'name': 'Warner Bros.', 'id': 6194}, {'name'...",1995-12-22
3,31357,Waiting to Exhale,"[{'id': 35, 'name': 'Comedy'}, {'id': 18, 'nam...","Cheated on, mistreated and stepped on, the wom...",[{'name': 'Twentieth Century Fox Film Corporat...,1995-12-22
4,11862,Father of the Bride Part II,"[{'id': 35, 'name': 'Comedy'}]",Just when George Banks has recovered from his ...,"[{'name': 'Sandollar Productions', 'id': 5842}...",1995-02-10


In [ ]:
#ID 전처리

new_id = []
for id in meta['id']:
    try:
        a = int(id)
        new_id.append(a)
    except:
        new_id.append(0) 
 
meta['id'] = new_id # id를 int로 변환 및 연도 값 들어간 경우 0으로 변환

In [ ]:
#Genre와 release_date 전처리

import re
meta['genres'] = meta['genres'].apply(lambda x: ' '.join(re.findall(r"\: '([A-z]+)\'", x))) # genre 딕셔너리 각각 꺼내기
meta['release_date'] = meta['release_date'].apply(lambda x: str(x)[:3] + '0').replace('nan0', np.nan) # release 십 년 단위로 가공하기

In [ ]:
#Production companies 전처리

from ast import literal_eval 

# Nan값과 []에 'name'을 key로 하는 dict로 채워주고, str을 dict으로 바꿔주기
meta['production_companies'] = meta['production_companies'].fillna("[{'name': ''}]").replace('[]', "[{'name': ''}]").apply(lambda x: str(x)).apply(literal_eval)

# production_companies name을 new_pc list에 할당하기
new_pc = []

for i in meta['production_companies']:       
    try:
        sub = []
        for j in range(len (i)):
            sub.append(i[j]['name'])
        new_pc.append(' '.join(sub))
    except:
        new_pc.append('')

new_pc

# 원래 열에 업데이트
meta['production_companies'] = new_pc

In [ ]:
meta.head()

,id,title,genres,overview,production_companies,release_date
0,862,Toy Story,Animation Comedy Family,"Led by Woody, Andy's toys live happily in his ...",Pixar Animation Studios,1990
1,8844,Jumanji,Adventure Fantasy Family,When siblings Judy and Peter discover an encha...,TriStar Pictures Teitler Film Interscope Commu...,1990
2,15602,Grumpier Old Men,Romance Comedy,A family wedding reignites the ancient feud be...,Warner Bros. Lancaster Gate,1990
3,31357,Waiting to Exhale,Comedy Drama Romance,"Cheated on, mistreated and stepped on, the wom...",Twentieth Century Fox Film Corporation,1990
4,11862,Father of the Bride Part II,Comedy,Just when George Banks has recovered from his ...,Sandollar Productions Touchstone Pictures,1990


In [ ]:
credit_df = pd.read_csv('/content/drive/MyDrive/SSAC/메인 프로젝트/Dataset/credits.csv')
credit_df    #데이터 확인 해보니 결측값 대신 [] 대신 들어가 있다

,cast,crew,id
0,"[{'cast_id': 14, 'character': 'Woody (voice)',...","[{'credit_id': '52fe4284c3a36847f8024f49', 'de...",862
1,"[{'cast_id': 1, 'character': 'Alan Parrish', '...","[{'credit_id': '52fe44bfc3a36847f80a7cd1', 'de...",8844
2,"[{'cast_id': 2, 'character': 'Max Goldman', 'c...","[{'credit_id': '52fe466a9251416c75077a89', 'de...",15602
3,"[{'cast_id': 1, 'character': ""Savannah 'Vannah...","[{'credit_id': '52fe44779251416c91011acb', 'de...",31357
4,"[{'cast_id': 1, 'character': 'George Banks', '...","[{'credit_id': '52fe44959251416c75039ed7', 'de...",11862
...,...,...,...
45471,"[{'cast_id': 0, 'character': '', 'credit_id': ...","[{'credit_id': '5894a97d925141426c00818c', 'de...",439050
45472,"[{'cast_id': 1002, 'character': 'Sister Angela...","[{'credit_id': '52fe4af1c3a36847f81e9b15', 'de...",111109
45473,"[{'cast_id': 6, 'character': 'Emily Shaw', 'cr...","[{'credit_id': '52fe4776c3a368484e0c8387', 'de...",67758
45474,"[{'cast_id': 2, 'character': '', 'credit_id': ...","[{'credit_id': '533bccebc3a36844cf0011a7', 'de...",227506


In [ ]:
#[] 있는 Row의 인덱스를 담는 리스트를 생성하여 해당 Row들을 제거해준다
credit_df.drop(list(credit_df[credit_df['cast'] == '[]'].index),inplace = True)
credit_df.drop(list(credit_df[credit_df['crew'] == '[]'].index),inplace = True)
credit_df

,cast,crew,id
0,"[{'cast_id': 14, 'character': 'Woody (voice)',...","[{'credit_id': '52fe4284c3a36847f8024f49', 'de...",862
1,"[{'cast_id': 1, 'character': 'Alan Parrish', '...","[{'credit_id': '52fe44bfc3a36847f80a7cd1', 'de...",8844
2,"[{'cast_id': 2, 'character': 'Max Goldman', 'c...","[{'credit_id': '52fe466a9251416c75077a89', 'de...",15602
3,"[{'cast_id': 1, 'character': ""Savannah 'Vannah...","[{'credit_id': '52fe44779251416c91011acb', 'de...",31357
4,"[{'cast_id': 1, 'character': 'George Banks', '...","[{'credit_id': '52fe44959251416c75039ed7', 'de...",11862
...,...,...,...
45470,"[{'cast_id': 1, 'character': 'Sir Robert Hode'...","[{'credit_id': '52fe44439251416c9100a899', 'de...",30840
45471,"[{'cast_id': 0, 'character': '', 'credit_id': ...","[{'credit_id': '5894a97d925141426c00818c', 'de...",439050
45472,"[{'cast_id': 1002, 'character': 'Sister Angela...","[{'credit_id': '52fe4af1c3a36847f81e9b15', 'de...",111109
45473,"[{'cast_id': 6, 'character': 'Emily Shaw', 'cr...","[{'credit_id': '52fe4776c3a368484e0c8387', 'de...",67758


In [ ]:
# 배우 상위 5명 리스트로 묶음
# 배우 전처리 과정
# 1. 각 셀마다 출연 배우 리스트의 이름만 뽑는다 (딕셔너리안에 'name'에 해당하는 부분)
# 2. 배우의 성과 이름만 가져와서 (중간이름 X) 사이 띄어쓰기를 없엔다 (띄어쓰기 대신 - 있는 경우도 있어서 따로 제거)
# 3. 그렇게 뽑은 배우 이름들 중 상위 5명만 하나의 문장 형식으로 만든다
credit_df['cast_list'] = credit_df['cast'].apply(lambda k : " ".join([y.split(' ')[0]+ y.split(' ')[-1]for y in [x.split("', 'order'")[0] for x in k.split("'name': '")[1:6]]]))
credit_df

,cast,crew,id,cast_list
0,"[{'cast_id': 14, 'character': 'Woody (voice)',...","[{'credit_id': '52fe4284c3a36847f8024f49', 'de...",862,TomHanks TimAllen DonRickles JimVarney Wallace...
1,"[{'cast_id': 1, 'character': 'Alan Parrish', '...","[{'credit_id': '52fe44bfc3a36847f80a7cd1', 'de...",8844,RobinWilliams JonathanHyde KirstenDunst Bradle...
2,"[{'cast_id': 2, 'character': 'Max Goldman', 'c...","[{'credit_id': '52fe466a9251416c75077a89', 'de...",15602,WalterMatthau JackLemmon Ann-MargretAnn-Margre...
3,"[{'cast_id': 1, 'character': ""Savannah 'Vannah...","[{'credit_id': '52fe44779251416c91011acb', 'de...",31357,WhitneyHouston AngelaBassett LorettaDevine Lel...
4,"[{'cast_id': 1, 'character': 'George Banks', '...","[{'credit_id': '52fe44959251416c75039ed7', 'de...",11862,SteveMartin DianeKeaton MartinShort KimberlyWi...
...,...,...,...,...
45470,"[{'cast_id': 1, 'character': 'Sir Robert Hode'...","[{'credit_id': '52fe44439251416c9100a899', 'de...",30840,PatrickBergin UmaThurman DavidMorrissey Jürgen...
45471,"[{'cast_id': 0, 'character': '', 'credit_id': ...","[{'credit_id': '5894a97d925141426c00818c', 'de...",439050,LeilaHatami KouroshTahami ElhamKorda
45472,"[{'cast_id': 1002, 'character': 'Sister Angela...","[{'credit_id': '52fe4af1c3a36847f81e9b15', 'de...",111109,AngelAquino PerryDizon HazelOrencio JoelTorre ...
45473,"[{'cast_id': 6, 'character': 'Emily Shaw', 'cr...","[{'credit_id': '52fe4776c3a368484e0c8387', 'de...",67758,ErikaEleniak AdamBaldwin JuliePage JamesRemar ...


In [ ]:
from ast import literal_eval

# crew : 감독 이름
for col in ['crew']:
    credit_df[col] = credit_df[col].apply(literal_eval) #딕셔너리 형식으로 쓰인 문자열을 진짜 딕셔너리로 형변환 해준다
    credit_df['director'] = credit_df[col].apply(lambda x : [d['name'] for d in x if d['job']=='Director']).apply(lambda x : " ".join(x)) #그 중 '디렉터'에 해당하는 부분과 그 감독 이름만 추출하여
    # credit_df['director'] = credit_df['director'].apply(lambda x : x.replace(' ','').replace('.','')) # 띄어쓰기를 제거해준다
     
credit_df['director'] = credit_df['director'].apply(lambda x : x.split(' ')[0]+x.split(' ')[-1]) #감독 이름 중 이름과 성만 가져와서 붙임 (중간이름 뺌)

In [ ]:
credit_df

,cast,crew,id,cast_list,director
0,"[{'cast_id': 14, 'character': 'Woody (voice)',...","[{'credit_id': '52fe4284c3a36847f8024f49', 'de...",862,TomHanks TimAllen DonRickles JimVarney Wallace...,JohnLasseter
1,"[{'cast_id': 1, 'character': 'Alan Parrish', '...","[{'credit_id': '52fe44bfc3a36847f80a7cd1', 'de...",8844,RobinWilliams JonathanHyde KirstenDunst Bradle...,JoeJohnston
2,"[{'cast_id': 2, 'character': 'Max Goldman', 'c...","[{'credit_id': '52fe466a9251416c75077a89', 'de...",15602,WalterMatthau JackLemmon Ann-MargretAnn-Margre...,HowardDeutch
3,"[{'cast_id': 1, 'character': ""Savannah 'Vannah...","[{'credit_id': '52fe44779251416c91011acb', 'de...",31357,WhitneyHouston AngelaBassett LorettaDevine Lel...,ForestWhitaker
4,"[{'cast_id': 1, 'character': 'George Banks', '...","[{'credit_id': '52fe44959251416c75039ed7', 'de...",11862,SteveMartin DianeKeaton MartinShort KimberlyWi...,CharlesShyer
...,...,...,...,...,...
45470,"[{'cast_id': 1, 'character': 'Sir Robert Hode'...","[{'credit_id': '52fe44439251416c9100a899', 'de...",30840,PatrickBergin UmaThurman DavidMorrissey Jürgen...,JohnIrvin
45471,"[{'cast_id': 0, 'character': '', 'credit_id': ...","[{'credit_id': '5894a97d925141426c00818c', 'de...",439050,LeilaHatami KouroshTahami ElhamKorda,HamidNematollah
45472,"[{'cast_id': 1002, 'character': 'Sister Angela...","[{'credit_id': '52fe4af1c3a36847f81e9b15', 'de...",111109,AngelAquino PerryDizon HazelOrencio JoelTorre ...,LavDiaz
45473,"[{'cast_id': 6, 'character': 'Emily Shaw', 'cr...","[{'credit_id': '52fe4776c3a368484e0c8387', 'de...",67758,ErikaEleniak AdamBaldwin JuliePage JamesRemar ...,MarkLester


In [ ]:
credit_df['character_list'] = credit_df['cast'].apply(lambda k : [x.replace('-','').replace(' ','').replace('/', ' ') for x in [x.split("', 'credit_id':")[0] for x in k.split("'character': '")][1:6]])
credit_df['character_list'] = credit_df['character_list'].apply(lambda k : " ".join(k))
credit_df

,cast,crew,id,cast_list,director,character_list
0,"[{'cast_id': 14, 'character': 'Woody (voice)',...","[{'credit_id': '52fe4284c3a36847f8024f49', 'de...",862,TomHanks TimAllen DonRickles JimVarney Wallace...,JohnLasseter,Woody(voice) BuzzLightyear(voice) Mr.PotatoHea...
1,"[{'cast_id': 1, 'character': 'Alan Parrish', '...","[{'credit_id': '52fe44bfc3a36847f80a7cd1', 'de...",8844,RobinWilliams JonathanHyde KirstenDunst Bradle...,JoeJohnston,AlanParrish SamuelAlanParrish VanPelt JudyShep...
2,"[{'cast_id': 2, 'character': 'Max Goldman', 'c...","[{'credit_id': '52fe466a9251416c75077a89', 'de...",15602,WalterMatthau JackLemmon Ann-MargretAnn-Margre...,HowardDeutch,MaxGoldman JohnGustafson ArielGustafson MariaS...
3,"[{'cast_id': 1, 'character': ""Savannah 'Vannah...","[{'credit_id': '52fe44779251416c91011acb', 'de...",31357,WhitneyHouston AngelaBassett LorettaDevine Lel...,ForestWhitaker,RobinStokes MarvinKing KennethDawkins JohnHarr...
4,"[{'cast_id': 1, 'character': 'George Banks', '...","[{'credit_id': '52fe44959251416c75039ed7', 'de...",11862,SteveMartin DianeKeaton MartinShort KimberlyWi...,CharlesShyer,GeorgeBanks NinaBanks FranckEggelhoffer AnnieB...
...,...,...,...,...,...,...
45470,"[{'cast_id': 1, 'character': 'Sir Robert Hode'...","[{'credit_id': '52fe44439251416c9100a899', 'de...",30840,PatrickBergin UmaThurman DavidMorrissey Jürgen...,JohnIrvin,SirRobertHode MaidMarian LittleJohn SirMilesFo...
45471,"[{'cast_id': 0, 'character': '', 'credit_id': ...","[{'credit_id': '5894a97d925141426c00818c', 'de...",439050,LeilaHatami KouroshTahami ElhamKorda,HamidNematollah,
45472,"[{'cast_id': 1002, 'character': 'Sister Angela...","[{'credit_id': '52fe4af1c3a36847f81e9b15', 'de...",111109,AngelAquino PerryDizon HazelOrencio JoelTorre ...,LavDiaz,SisterAngela Homer CrazyWoman Virgin AmangTibu...
45473,"[{'cast_id': 6, 'character': 'Emily Shaw', 'cr...","[{'credit_id': '52fe4776c3a368484e0c8387', 'de...",67758,ErikaEleniak AdamBaldwin JuliePage JamesRemar ...,MarkLester,EmilyShaw Det.MarkWinston JayneFerré AlexTyler...


In [ ]:
credit = credit_df[['id', 'cast_list', 'character_list', 'director']]
credit

,id,cast_list,character_list,director
0,862,TomHanks TimAllen DonRickles JimVarney Wallace...,Woody(voice) BuzzLightyear(voice) Mr.PotatoHea...,JohnLasseter
1,8844,RobinWilliams JonathanHyde KirstenDunst Bradle...,AlanParrish SamuelAlanParrish VanPelt JudyShep...,JoeJohnston
2,15602,WalterMatthau JackLemmon Ann-MargretAnn-Margre...,MaxGoldman JohnGustafson ArielGustafson MariaS...,HowardDeutch
3,31357,WhitneyHouston AngelaBassett LorettaDevine Lel...,RobinStokes MarvinKing KennethDawkins JohnHarr...,ForestWhitaker
4,11862,SteveMartin DianeKeaton MartinShort KimberlyWi...,GeorgeBanks NinaBanks FranckEggelhoffer AnnieB...,CharlesShyer
...,...,...,...,...
45470,30840,PatrickBergin UmaThurman DavidMorrissey Jürgen...,SirRobertHode MaidMarian LittleJohn SirMilesFo...,JohnIrvin
45471,439050,LeilaHatami KouroshTahami ElhamKorda,,HamidNematollah
45472,111109,AngelAquino PerryDizon HazelOrencio JoelTorre ...,SisterAngela Homer CrazyWoman Virgin AmangTibu...,LavDiaz
45473,67758,ErikaEleniak AdamBaldwin JuliePage JamesRemar ...,EmilyShaw Det.MarkWinston JayneFerré AlexTyler...,MarkLester


In [ ]:
keywords_df = pd.read_csv('/content/drive/MyDrive/SSAC/메인 프로젝트/Dataset/keywords.csv')
keywords_df

,id,keywords
0,862,"[{'id': 931, 'name': 'jealousy'}, {'id': 4290,..."
1,8844,"[{'id': 10090, 'name': 'board game'}, {'id': 1..."
2,15602,"[{'id': 1495, 'name': 'fishing'}, {'id': 12392..."
3,31357,"[{'id': 818, 'name': 'based on novel'}, {'id':..."
4,11862,"[{'id': 1009, 'name': 'baby'}, {'id': 1599, 'n..."
...,...,...
46414,439050,"[{'id': 10703, 'name': 'tragic love'}]"
46415,111109,"[{'id': 2679, 'name': 'artist'}, {'id': 14531,..."
46416,67758,[]
46417,227506,[]


In [ ]:
# []으로 된 사실상 결측치를 제거해줌
keywords_df.drop(list(keywords_df[keywords_df['keywords'] == '[]'].index), inplace = True)

In [ ]:
#credit과 마찬가지로 키워드에 해당하는 부분만 뽑아 하나의 문장으로 만든다
#대신 키워드끼리의 구분은 스페이스
#스페이스가 들어가있는 키워드를 하나로 합치는 버전이랑 그대로 둔 버전을 만든다
# based on novel이 키워드면 하나는 원래대로 두고 하나는 basedonnovel로 만듬
keywords_df['keyword_list'] = keywords_df['keywords'].apply(lambda k : " ".join([x.split("'}")[0] for x in k.split("'name': '")[1:]]))
keywords_df['keyword_list_no_space'] = keywords_df['keywords'].apply(lambda k : " ".join([x.split("'}")[0].replace(" ","") for x in k.split("'name': '")[1:]]))
keywords_df

,id,keywords,keyword_list,keyword_list_no_space
0,862,"[{'id': 931, 'name': 'jealousy'}, {'id': 4290,...",jealousy toy boy friendship friends rivalry bo...,jealousy toy boy friendship friends rivalry bo...
1,8844,"[{'id': 10090, 'name': 'board game'}, {'id': 1...",board game disappearance new home recluse gian...,boardgame disappearance newhome recluse gianti...
2,15602,"[{'id': 1495, 'name': 'fishing'}, {'id': 12392...",fishing best friend duringcreditsstinger old men,fishing bestfriend duringcreditsstinger oldmen
3,31357,"[{'id': 818, 'name': 'based on novel'}, {'id':...",based on novel interracial relationship single...,basedonnovel interracialrelationship singlemot...
4,11862,"[{'id': 1009, 'name': 'baby'}, {'id': 1599, 'n...",baby midlife crisis confidence aging daughter ...,baby midlifecrisis confidence aging daughter m...
...,...,...,...,...
46409,84419,"[{'id': 9748, 'name': 'revenge'}, {'id': 9826,...",revenge murder serial killer new york city scu...,revenge murder serialkiller newyorkcity sculpt...
46410,390959,"[{'id': 224180, 'name': 'blair witch'}]",blair witch,blairwitch
46411,289923,"[{'id': 616, 'name': 'witch'}, {'id': 2035, 'n...",witch mythology legend serial killer mockumentary,witch mythology legend serialkiller mockumentary
46414,439050,"[{'id': 10703, 'name': 'tragic love'}]",tragic love,tragiclove


In [ ]:
keyword = keywords_df[['id', 'keyword_list', 'keyword_list_no_space']]

In [ ]:
data_no_keyword = pd.merge(credit, meta, on = 'id')

In [ ]:
data_keyword = pd.merge(data_no_keyword, keyword, on='id')

In [ ]:
data_no_keyword.drop_duplicates(inplace=True)

In [ ]:
data_keyword.drop_duplicates(inplace=True)

In [ ]:
 data_no_keyword.to_csv('/content/drive/MyDrive/SSAC/메인 프로젝트/Dataset/data_no_keyword.csv', index=False)
 data_keyword.to_csv('/content/drive/MyDrive/SSAC/메인 프로젝트/Dataset/data_keyword.csv', index=False)